## OBSIC data correction

### Introduction

This Jupyter notebook demonstrates the tilt- and compliance-corrections of OBSIC data using the metadata included in the OBSIC metrics hub (http://obsic.whoi.edu).

The vertical (Z) data are corrected once for tilt and then corrected once for compliance.  Tilt corrections are made in the time domain by subtracting the predicted tilt noise from the raw data. The tilt model is parameterized in terms of a direction and angle of tilt for each deployment day; the compliance model is defined as a frequency-dependent complex transfer function for non response-corrected P and Z data:

\begin{align}
Z_{corrected} = Z_{raw} - Z_{predicted,tilt} - Z_{predicted,compliance}
\end{align}

A description of the derivation of the tilt and compliance models is given in http://obsic.whoi.edu.

### Assumptions

- The instrument response of the three seismometer components is assumed to be identical.
- Channel conventions follow the OBSIC left-hand standard: +Z-up, +H2 90 degrees clockwise with respect to +H1.
- Tilt model is defined daily.
- Compliance transfer function is an average across the entire deployment time and is in units of raw counts.
- Tilt correction is applied first.
- Tilt corrections are only provided for instrument days with working Z, H1 and H2.
- Compliance corrections are only provided for instruments with working Z, and pressure channels.
- This code downloads the tilt correciton only for the start time of the data
- Compliance transfer function is tapered to zero admittance at limits of infragravity wave window
- This notebook adds the tilt-corrected data as component *HT and the compliance-corrected data as *HC.  Non-standard.

In [ ]:
## functions
def hrotate (n_in, e_in, rotang):
    """
    Rotates the n_in and e_in (or 1 and 2) by rotang degrees.
            
    Assumes left-handed coordinate system (2 90 deg CW from 1)
    and a clockwise rotation from N.
    
    Angle  n_out  e_out (for testing)
    0     1    2
    90    -2   1
    -90   2   -1
            
    Parameters
    ----------
    rotang: float
        Rotation angle, in degrees, clockwise from N
    n_in: ndarray
        North (or 1) horizontal component
        East (or 2) horizontal component

    Returns
    -------
    ndarray, ndarray
        N and E data
    """
    c, s = np.cos(rotang/180*np.pi), np.sin(rotang/180*np.pi)  # rotn. matrix

    [n_out, e_out]=np.dot(np.array(((c, -s), (s, c))), np.array((n_in, e_in)))

    return (n_out, e_out)

def fetch_tilt_correction(netID, station, start_time, end_time, url_path):
    """fetches the tilt correction"""

    # read the tilt correction file
    df_tilt = pd.read_csv(f'{url_path}/{netID}/{station}.txt', skiprows=1, parse_dates=['Day'])

    # extract the tilt direction and angle for the startpoint of the requested data
    nearest_i = (df_tilt['Day'] - pd.Timestamp(start_time.datetime)).abs().idxmin()
    (tilt_direction,tilt_angle)=df_tilt.loc[nearest_i][['Tilt Direction', 'Tilt Angle']].values

    return tilt_direction, tilt_angle

def apply_tilt_correction(st, tilt_direction, tilt_angle):
    """applies the tilt correction to the stream
    :param st: stream object
    :param tilt_direction: direction of tilt
    :param tilt_angle: angle of tilt
    :return: stream object with tilt corrected traces, with channel name 'HHT'

    Assumes that the horizontal components are named *1 and *2, and the vertical component is named *Z
    Assumes that the tilt correction is a simple rotation of the horizontal components
    """

    # rotate the horizontal components into the direction of maximum tilt
    rotated_H, _ = hrotate(st.select(channel='*1')[0].data, st.select(channel='*2')[0].data,
                            tilt_direction)

    # subtract the predicted tilt noise from the uncorrected vertical component
    tilt_corrected_Z = st.select(channel='*Z')[0].data - rotated_H * np.tan(tilt_angle/180*np.pi)

    # add tilt-corrected trace to stream, with channel name 'HHT'
    tr = st.select(channel='*Z').copy()
    tr[0].data = tilt_corrected_Z
    tr[0].stats.channel='HHT'
    st = st + tr

    return st


def fetch_compliance_correction(netID, station, url_path):
    """fetches the compliance correction"""
    
    df_compliance=pd.read_csv(f'{url_path}/{netID}_{station}.txt', header=1, \
                               dtype = {'Frequency(Hz)': float, 'Transfer Function': str})
    freqs = df_compliance['Frequency(Hz)'].values
    tfs = df_compliance['Transfer Function'].apply(complex).values  # need to force from string to complex
    return freqs, tfs

def apply_compliance_correction(st, chan_Z, chan_P, freqs, tfs, taperspecs_f):
    """implement compliance correction to stream"""
    
    sps = st[0].stats.sampling_rate
    uncorrected_Z = st.select(channel=chan_Z)[0].data
    raw_P = st.select(channel=chan_P)[0].data

    seismic_fft = rfft(uncorrected_Z)
    pressure_fft = rfft(raw_P)
    freq_vector = rfftfreq(len(uncorrected_Z), d=1/sps)

    # interpolate the transfer function to the frequency vector
    TF_for_FFT = np.interp(freq_vector, freqs, tfs)

    predicted_Zc = irfft(pressure_fft * TF_for_FFT)
    corrected_Zc = uncorrected_Z - predicted_Zc

    # apply cosine taper
    #taperspecs_f = [freqs[0]-0.002, freqs[0], freqs[-1], freqs[-1]+0.01]
    costaper = cosine_taper(npts=len(freq_vector), p=0.5, halfcosine=True, freqs = freq_vector, flimit=taperspecs_f)

    # apply the taper to the transfer function
    TF_for_FFT_tapered=TF_for_FFT * costaper

    predicted_Zc_tapered = irfft(pressure_fft * TF_for_FFT_tapered)
    corrected_Zc_tapered = uncorrected_Z - predicted_Zc_tapered

    compliance_corrected_Z = corrected_Zc_tapered

    # add tilt-corrected trace to stream, with channel name 'HHT'
    tr = st.select(channel='*Z').copy()
    tr[0].data = compliance_corrected_Z
    tr[0].stats.channel='HHC'
    st = st + tr
    
    return st

In [12]:

import numpy as np
import pandas as pd
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

client = Client('IRIS')

# -- specify data and download from DMC
(netID, station)=('8Q', 'P06')
start_time = UTCDateTime('2020-04-10T00:01:00')
end_time = UTCDateTime('2020-04-10T01:01:00')
st = client.get_waveforms(netID, station, '', '*H*', start_time, end_time).merge().detrend()

# fetch the correction from the obsic website
#url_path=f'https://obsic-metrics.whoi.edu/static/images/tilt/{netID}/{station}.txt'
url_path=f'/Users/andrew/Desktop/metrics/transfer/tilt/daily_results_text/{netID}_{station}.txt'

# fetch the tilt correction for this station and day
tilt_direction, tilt_angle = fetch_tilt_correction(netID, station, start_time, end_time, url_path)

# apply the tilt correciton ot the stream
st = apply_tilt_correction(st, tilt_direction, tilt_angle)

# -- fetch the compliance correction --
url_path=f'/Users/andrew/Desktop/metrics/transfer/compliance/transfer_functions_text/'
freqs, tfs = fetch_compliance_correction(netID, station, url_path)

# apply the compliance correction to the stream
taperspecs_f = [0.002, 0.003, freqs_compliance[-1], freqs_compliance[-1]+0.01]
st = apply_compliance_correction(st, 'HHZ', 'HDH', freqs, tfs, taperspecs_f)